In [1]:
import pandas as pd
from pathlib import Path

In [11]:
# Get the data path
data_path = Path.cwd().parent.resolve()/ "src" / "hidden_debt_gsf" /"data"/"GFSMAB"/"GFSMAB2020"

# Define the file name
file_name = "GFSMAB2020_01-16-2025.csv"

# Combine the base path and file name
file_path = data_path / file_name

if file_path.exists():  # Check if the file exists
    data_2020 = pd.read_csv(file_path)
    print("File loaded successfully.")
else:
    print(f"File not found at {file_path}")


File loaded successfully.


In [16]:
# Extract head and save to CSV
headers = pd.DataFrame(data_2020.head(1000))
headers.to_csv("head_2020.csv", index=False)
print(f"Head saved")

test_2020 = data_2020[data_2020['Classification Code']=='GNLB|_Z']
print(len(data_2020), len(test_2020))

Head saved
540540 15967


In [14]:
# Get the data path
data_path = Path.cwd().parent.resolve()/ "src" / "hidden_debt_gsf" /"data"/"GFSMAB"/"GFSMAB2019"

# Define the file name
file_name = "GFSMAB2019_01-16-2025.csv"

# Combine the base path and file name
file_path = data_path / file_name

if file_path.exists():  # Check if the file exists
    data_2019 = pd.read_csv(file_path)
    print("File loaded successfully.")
else:
    print(f"File not found at {file_path}")


File loaded successfully.


In [22]:
# Extract head and save to CSV
headers = pd.DataFrame(data_2019.head(1000))
headers.to_csv("head_2019.csv", index=False)
print(f"Head saved")

test_2019 = data_2019[data_2019['Classification Name']=='Net lending (+) / Net borrowing (-)']
headers = pd.DataFrame(test_2019.head(1000))
headers.to_csv("head_test_2019.csv", index=False)
print(f"Head saved")

print(len(data_2019), len(test_2019))

Head saved
Head saved
508858 15022


In [13]:
test_2020 = data_2020[data_2020['Classification Code']=='GNLB|_Z']
print(len(data_2020), len(test_2020))

540540 15967


In [67]:
from pathlib import Path
import pandas as pd

# Define years of interest and base path
years = [2014, 2015, 2016, 2017, 2019, 2020]
base_path = Path.cwd().parent.resolve() / "src" / "hidden_debt_gsf" / "data" / "GFSMAB"

# Initialize an empty list to collect yearly data
merged_data = []

# Loop through the years and process each dataset
for year in years:
    file_dir = f"GFSMAB{year}"
    file_name = f"GFSMAB{year}_01-16-2025.csv"
    file_path = base_path / file_dir / file_name
    
    if file_path.exists():
        # Load the data for the year
        data = pd.read_csv(file_path)
        print(f"File for {year} loaded successfully.")
        
        # Add a vintage column
        data['Vintage'] = year
        
        # Append to the merged list
        merged_data.append(data)
    else:
        print(f"File not found for {year}: {file_path}")

# Combine all the filtered data into a single DataFrame
if merged_data:
    merged_df = pd.concat(merged_data, ignore_index=True)
    print("All data successfully merged.")
else:
    print("No data to merge.")

File for 2014 loaded successfully.
File for 2015 loaded successfully.
File for 2016 loaded successfully.
File for 2017 loaded successfully.
File for 2019 loaded successfully.
File for 2020 loaded successfully.
All data successfully merged.


In [7]:
print(len(merged_df))

1095602


In [11]:
from pathlib import Path
import pandas as pd

# Define keywords and base path
keywords = ["debt", "liabilities", "borrowing"]
years = [2014, 2015, 2016, 2017, 2019, 2020]
base_path = Path.cwd().parent.resolve() / "src" / "hidden_debt_gsf" / "data" / "GFSMAB"

def filter_and_combine(data, column_name, keywords):
    """
    Filter rows based on multiple keywords and return combined results 
    without duplicates.
    """
    combined_rows = pd.DataFrame()
    for keyword in keywords:
        # Filter rows matching the keyword
        filtered_rows = data[data[column_name].str.contains(keyword, case=False, na=False)].copy()
        combined_rows = pd.concat([combined_rows, filtered_rows], ignore_index=True)
    
    # Drop duplicate rows based on all columns except the 'Keyword' column
    combined_rows = combined_rows.drop_duplicates(subset=data.columns.tolist())
    
    return combined_rows

# Output file for the merged data
output_parquet = "filtered_merged_data.parquet"

# Initialize an empty list to collect filtered data
filtered_data_chunks = []

# Loop through the years and process each dataset in chunks
for year in years:
    file_dir = f"GFSMAB{year}"
    file_name = f"GFSMAB{year}_01-16-2025.csv"
    file_path = base_path / file_dir / file_name
    
    if file_path.exists():
        print(f"Processing file for {year}: {file_path}")
        
        # Process the file in chunks
        chunksize = 10_000
        for chunk in pd.read_csv(file_path, chunksize=chunksize):
            # Add a vintage column
            chunk['Vintage'] = year
            
            # Filter the chunk
            filtered_chunk = filter_and_combine(chunk, 'Classification Name', keywords)            
            # Append filtered chunk to the list
            filtered_data_chunks.append(filtered_chunk)
    else:
        print(f"File not found for {year}: {file_path}")

# Combine all filtered data and save as Parquet
if filtered_data_chunks:
    final_filtered_data = pd.concat(filtered_data_chunks, ignore_index=True)
    final_filtered_data.to_parquet(output_parquet, index=False)
    print(f"Filtered data saved to {output_parquet}")
else:
    print("No data to save.")


Processing file for 2014: /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSMAB/GFSMAB2014/GFSMAB2014_01-16-2025.csv
Processing file for 2015: /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSMAB/GFSMAB2015/GFSMAB2015_01-16-2025.csv
Processing file for 2016: /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSMAB/GFSMAB2016/GFSMAB2016_01-16-2025.csv
Processing file for 2017: /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSMAB/GFSMAB2017/GFSMAB2017_01-16-2025.csv
Processing file for 2019: /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSMAB/GFSMAB2019/GFSMAB2019_01-16-2025.csv
Processing file for 2020: /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSMAB/GFSMAB2020/GFSMAB2020_01-16-2025.csv
Filtered data saved to filtered_merged_data.parquet


In [16]:
import pandas as pd
merged_df = pd.read_parquet("filtered_merged_data.parquet")
# print(merged_df.head())

merged_test = merged_df[merged_df['Classification Name'].str.contains('D2', case=False, na=False)].copy()

print(merged_test.head())

              Country Name  Country Code  \
4760  Armenia, Republic of           911   
4761  Armenia, Republic of           911   
4762  Armenia, Republic of           911   
4763  Armenia, Republic of           911   
4764  Armenia, Republic of           911   

                       Classification Name Classification Code  \
4760  D2 debt liabilities at market value              G63|FD2   
4761  D2 debt liabilities at market value              G63|FD2   
4762  D2 debt liabilities at market value              G63|FD2   
4763  D2 debt liabilities at market value              G63|FD2   
4764  D2 debt liabilities at market value              G63|FD2   

                                           Sector Name Sector Code  \
4760  Central government (excl. social security funds)       S1311   
4761  Central government (excl. social security funds)       S1311   
4762  Central government (excl. social security funds)       S1311   
4763  Central government (excl. social security funds)    

In [2]:
import dask
print(dask.__version__)

2025.1.0


In [12]:
import pandas as pd

merged_df = pd.read_parquet('filtered_merged_data.parquet')

# Step 1: Initialize an empty list to store the most populated sector and classification data per country
most_populated_combinations = []

# Step 2: Get a list of unique country codes
unique_countries = merged_df['Country Code'].unique()

# Step 3: Iterate over each country and process its data
for country_code in unique_countries:
    # Filter data for the current country
    country_data = merged_df[
        (merged_df['Country Code'] == country_code) & 
        (merged_df['Attribute'] == 'Value') & 
        (merged_df['Unit Code'] == 'XDC') & # Domestic Currency
        (merged_df['Sector Code']!= 'S1312') & # State Governments
        (merged_df['Sector Code']!= 'S1313') & # Local Governments
        (merged_df['Sector Code']!= 'S1314') # Social security funds
    ]

    # Skip if no data for the country
    if country_data.empty:
        continue

    # Identify year columns (1970 to 2020)
    year_columns = [col for col in country_data.columns if col.isdigit() and 1970 <= int(col) <= 2020]

    # Identify the most populated combination of Sector Code and Classification Code for the current country
    most_data_entries = (
        country_data
        .groupby(['Sector Code', 'Classification Code'])[year_columns]
        .apply(lambda group: group.notna().sum().sum())
        .reset_index(name='Data Entry Count')
        .sort_values(by='Data Entry Count', ascending=False)
    )

    # Get the most populated combination
    if not most_data_entries.empty:
        most_populated_combination = most_data_entries.iloc[0][['Sector Code', 'Classification Code']]
        
        # Filter data for the most populated combination
        most_populated_combination_data = country_data[
            (country_data['Sector Code'] == most_populated_combination['Sector Code']) &
            (country_data['Classification Code'] == most_populated_combination['Classification Code'])
        ]
        
        # Append the result to the list
        most_populated_combinations.append(most_populated_combination_data)

# Step 4: Combine all the most populated combination data into a single DataFrame
combined_most_populated_combinations = pd.concat(most_populated_combinations, ignore_index=True)

# Save the combined data to a CSV file
combined_most_populated_combinations.to_csv("most_populated_combinations_by_country.csv", index=False)

# Display a message to indicate completion
print("The dataset with the most populated combination of sector and classification by country has been saved to 'most_populated_combinations_by_country.csv'.")


The dataset with the most populated combination of sector and classification by country has been saved to 'most_populated_combinations_by_country.csv'.


In [ ]:
import pandas as pd
import plotly.express as px

# Assuming merged_df is available
# Step 1: Filter the dataset
filtered_data = merged_df[
    (merged_df['Country Code'] == 122) & 
    (merged_df['Attribute'] == 'Value') & 
    (merged_df['Unit Code'] == 'XDC')
]

# Save filtered dataset
filtered_data.to_csv("austria_filtered.csv", index=False)

# Step 2: Identify year columns (1970 to 2020)
year_columns = [col for col in filtered_data.columns if col.isdigit() and 1970 <= int(col) <= 2020]

# Identify the most populated Sector Code
most_data_entries = (
    filtered_data
    .groupby('Sector Code')[year_columns]
    .apply(lambda group: group.notna().sum().sum())
    .reset_index(name='Data Entry Count')
    .sort_values(by='Data Entry Count', ascending=False)
)

# Get the most populated Sector Name
most_populated_sector = most_data_entries.iloc[0]['Sector Code']

filtered_data = filtered_data[filtered_data['Sector Code'] == most_populated_sector]


In [61]:
import pandas as pd

# Assuming merged_df is available
# Step 1: Filter the dataset
filtered_data = merged_df[
    (merged_df['Country Code'] == 122) & 
    (merged_df['Attribute'] == 'Value') & 
    (merged_df['Unit Code'] == 'XDC')
]

# Save filtered dataset
filtered_data.to_csv("austria_filtered.csv", index=False)

# Step 2: Identify year columns (1970 to 2020)
year_columns = [col for col in filtered_data.columns if col.isdigit() and 1970 <= int(col) <= 2020]

# Identify the most populated Sector Code
most_data_entries = (
    filtered_data
    .groupby('Sector Code')[year_columns]
    .apply(lambda group: group.notna().sum().sum())
    .reset_index(name='Data Entry Count')
    .sort_values(by='Data Entry Count', ascending=False)
)

# Get the most populated Sector Name
most_populated_sector = most_data_entries.iloc[0]['Sector Code']

filtered_data = filtered_data[filtered_data['Sector Code']==most_populated_sector]

# Step 3: Reshape data to pivot by vintage
pivot_data = (
    filtered_data.melt(
        id_vars=['Sector Name', 'Vintage'], 
        value_vars=year_columns, 
        var_name='Year', 
        value_name='Value'
    )
)

# Ensure 'Value' is numeric
pivot_data['Value'] = pd.to_numeric(pivot_data['Value'], errors='coerce')

# Step 4: Calculate differences for the same year across vintages
pivot_data['Year'] = pivot_data['Year'].astype(int)
difference_data = (
    pivot_data
    .sort_values(by=['Year', 'Vintage'])
    .groupby(['Sector Name', 'Year'], group_keys=False)
    .apply(lambda group: group.assign(Difference=group['Value'].diff()))
    .dropna(subset=['Difference'])
)

# Save results
difference_data.to_csv("austria_vintage_differences.csv", index=False)


/tmp/ipykernel_6380/1732521603.py:50: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [ ]:
import pandas as pd
import plotly.express as px

# Assuming merged_df is available
# Step 1: Filter the dataset
filtered_data = merged_df[
    (merged_df['Country Code'] == 122) & 
    (merged_df['Attribute'] == 'Value') & 
    (merged_df['Unit Code'] == 'XDC')
]

# Save filtered dataset
filtered_data.to_csv("austria_filtered.csv", index=False)

# Step 2: Identify year columns (1970 to 2020)
year_columns = [col for col in filtered_data.columns if col.isdigit() and 1970 <= int(col) <= 2020]

# Identify the most populated Sector Code
most_data_entries = (
    filtered_data
    .groupby('Sector Code')[year_columns]
    .apply(lambda group: group.notna().sum().sum())
    .reset_index(name='Data Entry Count')
    .sort_values(by='Data Entry Count', ascending=False)
)

# Get the most populated Sector Name
most_populated_sector = most_data_entries.iloc[0]['Sector Code']

filtered_data = filtered_data[filtered_data['Sector Code']==most_populated_sector]

# Step 3: Reshape data to pivot by vintage
pivot_data = (
    filtered_data.melt(
        id_vars=['Sector Name', 'Vintage'], 
        value_vars=year_columns, 
        var_name='Year', 
        value_name='Value'
    )
)

# Ensure 'Value' is numeric
pivot_data['Value'] = pd.to_numeric(pivot_data['Value'], errors='coerce')

# Step 4: Calculate differences for the same year across vintages
pivot_data['Year'] = pivot_data['Year'].astype(int)
difference_data = (
    pivot_data
    .sort_values(by=['Year', 'Vintage'])
    .groupby(['Sector Name', 'Year'], group_keys=False)
    .apply(lambda group: group.assign(Difference=group['Value'].diff()))
    .dropna(subset=['Difference'])
)

# Assuming `pivot_data` is available and contains 'Year', 'Vintage', and 'Value' columns
# Ensure 'Year' and 'Vintage' are of the correct type
pivot_data['Year'] = pivot_data['Year'].astype(int)
pivot_data['Vintage'] = pivot_data['Vintage'].astype(str)

# Filter data to exclude NaN values for plotting
plot_data = pivot_data.dropna(subset=['Value'])

# Create the bar chart with grouped bars
fig = px.bar(
    plot_data,
    x='Year',
    y='Value',
    color='Vintage',
    barmode='group',  # Ensure bars are grouped side by side
    title='Yearly Values by Vintage',
    labels={'Year': 'Year', 'Value': 'Value', 'Vintage': 'Vintage'},
)

# Update layout for better readability
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Value',
    legend_title='Vintage',
    template='plotly_white',
    bargap=0.15,  # Adjust bar spacing for clarity
    bargroupgap=0.1  # Adjust group spacing
)

# Show the figure
fig.show()


          Sector Name Vintage  Year         Value
0  General government    2014  1972  5.094366e+08
1  General government    2015  1972  5.094366e+08
2  General government    2016  1972  5.094366e+08
3  General government    2017  1972  5.094366e+08
4  General government    2019  1972  5.094366e+08


/tmp/ipykernel_6380/1321573841.py:50: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [39]:
austria_merged = merged_df[merged_df['Country Code']==122]
print(len(austria_merged))
austria_merged = austria_merged[austria_merged['Attribute']=='Value']
print(len(austria_merged))
austria_merged = austria_merged[austria_merged['Unit Code']=='XDC']
print(len(austria_merged))
austria_merged.to_csv("austria_merged.csv", index=False)

# Identifying year columns (from 1970 to 2020) in the dataset
year_columns = [col for col in austria_merged.columns if col.isdigit() and 1970 <= int(col) <= 2020]

# Filtering rows with non-numeric values and counting non-NA numeric entries per combination
most_data_entries = (
    austria_merged
    .groupby('Sector Name')[year_columns]
    .apply(lambda group: group.notna().sum().sum())
    .reset_index(name='Data Entry Count')
    .sort_values(by='Data Entry Count', ascending=False)
)

most_data_entries.to_csv("austria_by_entries.csv")

528
96
48


In [ ]:
import pandas as pd

# Load the previously saved `austria_merged.csv` file
file_path = "austria_merged.csv"
austria_merged = pd.read_csv(file_path)

# Identifying year columns (from 1970 to 2020)
year_columns = [col for col in austria_merged.columns if col.isdigit() and 1970 <= int(col) <= 2020]

# Identify the most populated Sector Code
most_data_entries = (
    austria_merged
    .groupby('Sector Name')[year_columns]
    .apply(lambda group: group.notna().sum().sum())
    .reset_index(name='Data Entry Count')
    .sort_values(by='Data Entry Count', ascending=False)
)

# Get the most populated Sector Name
most_populated_sector = most_data_entries.iloc[0]['Sector Name']

# Filter data for the most populated Sector Code
filtered_data = austria_merged[austria_merged['Sector Name'] == most_populated_sector]

# Sort by vintage (chronological order)
filtered_data = filtered_data.sort_values(by='Vintage')

# Compute year-over-year changes for each year column
yearly_changes = filtered_data[year_columns].diff(axis=0)
yearly_changes.insert(0, 'Vintage', filtered_data['Vintage'].values)
yearly_changes.insert(1, 'Sector Name', filtered_data['Sector Name'].values)

# Save the year-over-year changes to a new CSV file
yearly_changes.to_csv("austria_yearly_changes.csv", index=False)

# Display the result to the user
import ace_tools as tools; tools.display_dataframe_to_user(name="Yearly Changes for Most Populated Sector", dataframe=yearly_changes)


In [32]:
# Identifying year columns (from 1970 to 2020) in the dataset
year_columns = [col for col in austria_merged.columns if col.isdigit() and 1970 <= int(col) <= 2020]

# Filtering rows with non-numeric values and counting non-NA numeric entries per combination
most_data_entries = (
    austria_merged
    .groupby(['Sector Code', 'Unit Code'])[year_columns]
    .apply(lambda group: group.notna().sum().sum())
    .reset_index(name='Data Entry Count')
    .sort_values(by='Data Entry Count', ascending=False)
)
most_data_entries.to_csv("austria_by_entries.csv")

In [3]:
# Columns to extract unique values from
columns_to_extract = ["Residence Name", "Instrument and Assets Classification Name"]

# Extract and save unique values for each column
for column in columns_to_extract:
    unique_values = data[column].dropna().unique()  # Drop NaN values and get unique ones
    unique_df = pd.DataFrame(unique_values, columns=[column])  # Convert to DataFrame
    output_file = f"unique_{column.replace(' ', '_')}.csv"  # Save file
    unique_df.to_csv(output_file, index=False)
    print(f"Unique values from '{column}' saved to {output_file}")

Headers saved


KeyError: 'Residence Name'

In [36]:
def filter_and_combine(data, column_name, keywords, output_filename):
    """
    Filters rows based on multiple keywords in a specified column and combines them into one CSV.
    
    Parameters:
        data (pd.DataFrame): The DataFrame to search within.
        column_name (str): The column to search for the keywords.
        keywords (list): A list of keywords to search for.
        output_filename (str): The filename to save the combined filtered rows.

    Returns:
        pd.DataFrame: Combined DataFrame of filtered rows.
    """
    combined_rows = pd.DataFrame()  # Initialize an empty DataFrame

    for keyword in keywords:
        # Filter rows where the keyword appears in the specified column (case-insensitive)
        filtered_rows = data[data[column_name].str.contains(keyword, case=False, na=False)]
        
        # Add a column to indicate the keyword used for filtering
        filtered_rows['Keyword'] = keyword
        
        # Append to the combined DataFrame
        combined_rows = pd.concat([combined_rows, filtered_rows], ignore_index=True)

    # Save the combined rows to a single CSV file
    combined_rows.to_csv(Path.cwd() / output_filename, index=False)
    
    # Return the combined DataFrame
    return combined_rows

# Define the DataFrame (assuming `data` is already loaded)
column_name = "Classification Name"

# List of keywords to search for
keywords = ["debt", "liabilities", "borrowing"]

# Call the function to filter and combine rows
output_filename = "filtered_rows_combined.csv"
combined_data = filter_and_combine(data, column_name, keywords, output_filename)

# Display the unique values in the "Classification Name" column
unique_classification_names = combined_data[column_name].unique()
print("Unique classification names:", unique_classification_names)

In [45]:
def analyze_data_format(data):
    """
    Analyzes the dataset to calculate the sum of legitimate entries and covered countries
    for each combination of Classification, Sector, and Unit.
    
    Parameters:
        data (pd.DataFrame): The input dataset.

    Returns:
        pd.DataFrame: Summary DataFrame with analysis results.
    """
    # Filter rows where Attribute is "Value"
    filtered_data = data[data['Attribute'] == 'Value']

    # Select year columns (assume all columns except metadata and Attribute are years)
    year_columns = [col for col in data.columns if col.startswith('20') or col.startswith('19')]

    # Group by Classification, Sector, and Unit
    grouped = filtered_data.groupby(
        ['Classification Name', 'Sector Name', 'Unit Name']
    )

    # Calculate the required summaries
    summary = grouped.apply(
        lambda group: pd.Series({
            'Sum of Legitimate Entries': group[year_columns].apply(
                pd.to_numeric, errors='coerce'
            ).count().sum(),
            'Number of Covered Countries': group['Country Code'].nunique()
        })
    ).reset_index()

    return summary

# Assuming your dataset is loaded in `data`
summary = analyze_data_format(combined_data)

# Save the summary to a CSV file if needed
summary.to_csv(Path.cwd().parent.resolve()/"data"/"GFSYMAB"/"Summaries"/"summary_analysis_2015.csv", index=False)


/tmp/ipykernel_66411/4276670487.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = grouped.apply(
